# Generate force coefficients

The following steps are performed to obtain all the artifacts for force coefficient use case:

1. Read parameters file
2. Read Mesh description
   * Its path can be specified
   * Or it can be obtained from the simulation configuration file
3. Read data for pressure coefficient

Instantiate path manager

In [5]:
from cfdmod.use_cases.pressure.path_manager import CfPathManager
import pathlib

path_manager = CfPathManager(output_path=pathlib.Path("./output/pressure"))

Read parameters file

In [6]:
from cfdmod.use_cases.pressure.force.Cf_config import CfCaseConfig

config_path = pathlib.Path("./fixtures/tests/pressure/Cf_params.yaml")
post_proc_cfg = CfCaseConfig.from_file(config_path)

post_proc_cfg.force_coefficient

{'measurement_1': CfConfig(statistics=[BasicStatisticModel(stats='mean', display_name=''), BasicStatisticModel(stats='rms', display_name=''), BasicStatisticModel(stats='skewness', display_name=''), BasicStatisticModel(stats='kurtosis', display_name=''), ParameterizedStatisticModel(stats='mean_eq', display_name='', params=MeanEquivalentParamsModel(scale_factor=0.61)), ParameterizedStatisticModel(stats='min', display_name='', params=ExtremeMovingAverageParamsModel(method_type='Moving Average', window_size_interval=3.0, full_scale_U_H=40.0, full_scale_characteristic_length=22.4)), ParameterizedStatisticModel(stats='max', display_name='', params=ExtremeAbsoluteParamsModel(method_type='Absolute'))], bodies=[BodyConfig(name='building2', sub_bodies=ZoningModel(x_intervals=[-inf, inf], y_intervals=[-inf, inf], z_intervals=[0.0, 10.0, 20.0]))], directions=['x', 'y', 'z'], transformation=TransformationConfig(translation=(0, 0, 0), rotation=(0.0, 0.0, 0.0), fixed_point=(0, 0, 0))),
 'measurement_

Read Mesh from file

In [7]:
from lnas import LnasFormat

mesh_path = pathlib.Path("./fixtures/tests/pressure/galpao/galpao.normalized.lnas")
mesh = LnasFormat.from_file(mesh_path)

mesh.geometry.vertices.shape, mesh.geometry.triangles.shape

((1549, 3), (2915, 3))

Calculate force coefficient

In [8]:
from cfdmod.logger import logger
from cfdmod.use_cases.pressure.force.Cf_data import CommonOutput, process_Cf

cp_path = pathlib.Path("./fixtures/tests/pressure/data/cp_t.normalized.h5")

for cfg_label, cfg in post_proc_cfg.force_coefficient.items():
        logger.info(f"Processing Cf config {cfg_label} ...")

        cf_output_dict: dict[str, CommonOutput] = process_Cf(
            mesh=mesh,
            cfg=cfg,
            cp_path=cp_path,
            bodies_definition=post_proc_cfg.bodies,
        )
        already_saved = False
        for direction_lbl, cf_output in cf_output_dict.items():
            path_manager.direction_label = direction_lbl
            if already_saved:
                cf_output.save_outputs(cfg_label=cfg_label, path_manager=path_manager)
            else:
                cf_output.save_region_info(cfg_label=cfg_label, path_manager=path_manager)
                cf_output.save_outputs(cfg_label=cfg_label, path_manager=path_manager)
                already_saved = True

        logger.info(f"Processed Cf config {cfg_label}!")


[2024-05-23 15:39:21,869] [INFO] - cfdmod - Processing Cf config measurement_1 ... (1997722511.py:7)
[2024-05-23 15:39:22,960] [INFO] - cfdmod - Processed Cf config measurement_1! (1997722511.py:25)
[2024-05-23 15:39:22,961] [INFO] - cfdmod - Processing Cf config measurement_2 ... (1997722511.py:7)
[2024-05-23 15:39:23,952] [INFO] - cfdmod - Processed Cf config measurement_2! (1997722511.py:25)
